In [77]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import random
from metrics import MAPE_GroupFairnessScorer, evaluate_predictions

Look into autogluon tutorial for explanations of this code  
The dtype of the columns determines the interpretation and preprocessing of the features  
Autogluon has its own dataframe type with static and dynamic features

In [78]:
df = pd.read_csv("../possible_datasets/M4/train.csv")
static_features_df = pd.read_csv("../possible_datasets/M4/metadata.csv")

static_features_df.domain.value_counts()

domain
Micro       43
Finance     36
Industry    11
Other        8
Macro        2
Name: count, dtype: int64

In [79]:
drop_ids = static_features_df.loc[static_features_df.domain == "Macro", "item_id"]
static_features_df = static_features_df.loc[~static_features_df.item_id.isin(drop_ids)]
df = df.loc[~df.item_id.isin(drop_ids)]

sampling strategy SMOTE needs later at least >= 5 samples to produce meaningful results so drop the items with static feature Macro

In [135]:
WEEKEND_INDICES = [5, 6]
df["weekend"] = pd.DatetimeIndex(df["timestamp"].astype('datetime64[ns]').values).weekday.isin(WEEKEND_INDICES)

train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_features_df,
)

PREDICTION_LENGTH = 14
train_data, test_data = train_data.train_test_split(PREDICTION_LENGTH)      #split seems to be done stratified with respect to the static features

Sorting the dataframe index before generating the train/test split.


# Baseline model

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=14,
    target="target",    #specify that target is the target and weekend a known active covariate, other dynamic features are automatically detected as known covariates
    known_covariates_names=["weekend"],
    #eval_metric is default WQL
).fit(train_data, presets="fast_training")

Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Luca\Studium\Master\Master-project\Autogluon_Timeseries\AutogluonModels\ag-20251204_071012'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          4
GPU Count:          0
Memory Avail:       1.83 GB / 7.88 GB (23.2%)
Disk Space Avail:   78.60 GB / 475.69 GB (16.5%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'very_light',
 'known_covariates_names': ['weekend'],
 'num_val_windows': 1,
 'prediction_length': 14,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'D'
Provided train_data has 242073 

In [ ]:
df_baseline = evaluate_predictions(test_data, predictor)
df_baseline

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


{'domain':         Industry     Finance       Micro      Other         std        cv  \
 RMSE -129.635232 -589.437311 -363.932133 -95.698247  229.863180 -0.780055   
 MAE   -78.494096 -272.632741 -174.433554 -55.402985   99.342637 -0.683986   
 MAPE   -0.023865   -0.029169   -0.031825  -0.020037    0.005288 -0.201648   
 
         max_diff   mean_diff  
 RMSE  493.739065  285.919016  
 MAE   217.229756  124.604787  
 MAPE    0.011788    0.006778  }

# Use Fairness Metric for evalauation

In [ ]:
#from metrics import MAPE_GroupFairnessScorer

predictor = TimeSeriesPredictor(
    prediction_length=14,
    target="target",    #specify that target is the target and weekend a known active covariate, other dynamic features are automatically detected as known covariates
    known_covariates_names=["weekend"],
    eval_metric=MAPE_GroupFairnessScorer()# train_data.static_features
).fit(train_data, presets="fast_training")

Beginning AutoGluon training...


AutoGluon will save models to 'c:\Users\Luca\Studium\Master\Master-project\Autogluon_Timeseries\AutogluonModels\ag-20251204_071519'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          4
GPU Count:          0
Memory Avail:       1.80 GB / 7.88 GB (22.8%)
Disk Space Avail:   78.48 GB / 475.69 GB (16.5%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE_GroupFairnessScorer,
 'hyperparameters': 'very_light',
 'known_covariates_names': ['weekend'],
 'num_val_windows': 1,
 'prediction_length': 14,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'D'
Provided train_data has 242073 rows, 98 ti

In [ ]:
df_regularization = evaluate_predictions(test_data, predictor)
df_regularization["domain"]

Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS


,Industry,Finance,Micro,Other,std,cv,max_diff,mean_diff
RMSE,-121.778391,-583.782426,-355.112093,-227.545166,198.860794,-0.617476,462.004035,252.263172
MAE,-73.513268,-267.781804,-168.981017,-100.387109,86.633453,-0.567471,194.268535,108.566586
MAPE,-0.022205,-0.028316,-0.030521,-0.029472,0.003726,-0.134859,0.008316,0.004351


In [ ]:
s_base = df_baseline["domain"].copy()
s_reg = df_regularization["domain"].copy()

s_base.index = s_base.index.astype(str) + "_baseline"
s_reg.index = s_reg.index.astype(str) + "_regularization"

pd.concat([s_base, s_reg], axis=0).sort_index()

,Industry,Finance,Micro,Other,std,cv,max_diff,mean_diff
MAE_baseline,-78.494096,-272.632741,-174.433554,-55.402985,99.342637,-0.683986,217.229756,124.604787
MAE_regularization,-73.513268,-267.781804,-168.981017,-100.387109,86.633453,-0.567471,194.268535,108.566586
MAPE_baseline,-0.023865,-0.029169,-0.031825,-0.020037,0.005288,-0.201648,0.011788,0.006778
MAPE_regularization,-0.022205,-0.028316,-0.030521,-0.029472,0.003726,-0.134859,0.008316,0.004351
RMSE_baseline,-129.635232,-589.437311,-363.932133,-95.698247,229.863180,-0.780055,493.739065,285.919016
RMSE_regularization,-121.778391,-583.782426,-355.112093,-227.545166,198.860794,-0.617476,462.004035,252.263172


# Use sampling strategy/synthetic data generation 

Simple resampling

In [170]:
df = pd.read_csv("../possible_datasets/M4/train.csv")
static_features_df = pd.read_csv("../possible_datasets/M4/metadata.csv")

drop_ids = static_features_df.loc[static_features_df.domain == "Macro", "item_id"]
static_features_df = static_features_df.loc[~static_features_df.item_id.isin(drop_ids)]
df = df.loc[~df.item_id.isin(drop_ids)]

WEEKEND_INDICES = [5, 6]
df["weekend"] = pd.DatetimeIndex(df["timestamp"].astype('datetime64[ns]').values).weekday.isin(WEEKEND_INDICES)

static_features_df.domain.value_counts()

domain
Micro       43
Finance     36
Industry    11
Other        8
Name: count, dtype: int64

In [181]:
random.seed(42)
ids_industry = random.choices(list(static_features_df[static_features_df.domain == "Industry"].item_id), k=19)
ids_other = random.choices(list(static_features_df[static_features_df.domain == "Other"].item_id), k=22)

Bring Other and Industry up to 30 datapoints

In [182]:
m4_oversampled = pd.DataFrame()
for e, id in enumerate(ids_industry + ids_other):
    m4_oversampled_id = df.loc[df.item_id == id].copy()
    m4_oversample_static = static_features_df.loc[static_features_df.item_id == id].copy()
    m4_oversampled_id.loc[:, "item_id"] += str(e)
    m4_oversample_static.loc[:, "item_id"] += str(e)
    static_features_df = pd.concat([static_features_df, m4_oversample_static], axis=0)
    df = pd.concat([df, m4_oversampled_id], axis=0)
static_features_df.domain.value_counts()

domain
Micro       43
Finance     36
Industry    30
Other       30
Name: count, dtype: int64

In [153]:
random.seed(42)
ids_industry = random.choices(list(test_data.static_features[test_data.static_features.domain == "Industry"].index), k=19)
ids_other = random.choices(list(test_data.static_features[test_data.static_features.domain == "Other"].index), k=22)

In [183]:
train_data_oversampling = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_features_df,
)

PREDICTION_LENGTH = 14
train_data_oversampling, test_data_oversampling = train_data_oversampling.train_test_split(PREDICTION_LENGTH) 

Sorting the dataframe index before generating the train/test split.


In [184]:
predictor = TimeSeriesPredictor(
    prediction_length=14,
    target="target",    #specify that target is the target and weekend a known active covariate, other dynamic features are automatically detected as known covariates
    known_covariates_names=["weekend"],
    #eval_metric is default WQL
).fit(train_data_oversampling, presets="fast_training")
df_oversampling = evaluate_predictions(test_data_oversampling, predictor)

Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Luca\Studium\Master\Master-project\Autogluon_Timeseries\AutogluonModels\ag-20251204_081449'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          4
GPU Count:          0
Memory Avail:       1.07 GB / 7.88 GB (13.6%)
Disk Space Avail:   79.94 GB / 475.69 GB (16.8%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'very_light',
 'known_covariates_names': ['weekend'],
 'num_val_windows': 1,
 'prediction_length': 14,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'D'
Provided train_data has 254183 

In [185]:
s_base = df_baseline["domain"].copy()
s_reg = df_regularization["domain"].copy()
s_over = df_oversampling["domain"].copy()

s_base.index = s_base.index.astype(str) + "_baseline"
s_reg.index = s_reg.index.astype(str) + "_regularization"
s_over.index = s_over.index.astype(str) + "_oversampling"

pd.concat([s_base, s_reg, s_over], axis=0).sort_index()

,Industry,Finance,Micro,Other,std,cv,max_diff,mean_diff
MAE_baseline,-78.494096,-272.632741,-174.433554,-55.402985,99.342637,-0.683986,217.229756,124.604787
MAE_oversampling,-60.848085,-272.787878,-176.162979,-77.655132,98.156097,-0.668349,211.939793,122.387871
MAE_regularization,-73.513268,-267.781804,-168.981017,-100.387109,86.633453,-0.567471,194.268535,108.566586
MAPE_baseline,-0.023865,-0.029169,-0.031825,-0.020037,0.005288,-0.201648,0.011788,0.006778
MAPE_oversampling,-0.022646,-0.029389,-0.031635,-0.024756,0.004129,-0.152314,0.008989,0.005267
MAPE_regularization,-0.022205,-0.028316,-0.030521,-0.029472,0.003726,-0.134859,0.008316,0.004351
RMSE_baseline,-129.635232,-589.437311,-363.932133,-95.698247,229.863180,-0.780055,493.739065,285.919016
RMSE_oversampling,-103.731981,-590.230865,-368.305097,-122.128443,230.215913,-0.777496,486.498884,284.278884
RMSE_regularization,-121.778391,-583.782426,-355.112093,-227.545166,198.860794,-0.617476,462.004035,252.263172


The baseline model is the worst model, oversampling the second best and regularization the best